In [103]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display, HTML
import numpy as np

# Get list of S&P 500 symbols
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
top_symbols = df['Symbol'].tolist()[:5]  # Limiting to 50 for demonstration

# Set target date (45 days from now)
target_date = datetime.now() + timedelta(days=45)

In [104]:
# Function to find closest expiration date
def find_closest_expiration(expiration_dates, target_date):
    return min(expiration_dates, key=lambda x: abs(datetime.strptime(x, "%Y-%m-%d") - target_date))

# Calculate IVR
def calculate_ivr(symbol, current_iv):
    try:
        stock = yf.Ticker(symbol)
        hist_volatility = stock.history(period="1y")['Close'].pct_change().std() * np.sqrt(252) * 100
        
        if hist_volatility.max() == hist_volatility.min():
            # If historical volatility is constant, set IVR to 50%
            ivr = 50
        else:
            ivr = (current_iv - hist_volatility.min()) / (hist_volatility.max() - hist_volatility.min()) * 100
        
        return round(ivr, 2)
    except Exception as e:
        print(f"Error calculating IVR for {symbol}: {str(e)}")
        return np.nan

In [105]:
results = []

# Loop through symbols and get options data
for symbol in top_symbols:
    try:
        print(f"Processing {symbol}")
        
        # Create Ticker object
        ticker = yf.Ticker(symbol)
        
        # Get expiration dates
        exp_dates = ticker.options
        
        if not exp_dates:
            print(f"No expiration dates found for {symbol}")
            continue
        
        # Find closest expiration date to target
        closest_exp = find_closest_expiration(exp_dates, target_date)
        print(f"Closest expiration for {symbol}: {closest_exp}")
        
        # Get options chain for closest expiration
        opt = ticker.option_chain(closest_exp.strftime("%Y-%m-%d"))
        
        if opt.calls.empty or opt.puts.empty:
            print(f"No options data for {symbol}")
            continue
        
        # Get stock price
        stock_price = ticker.info.get('regularMarketPrice')
        if stock_price is None:
            print(f"No current price available for {symbol}")
            continue
        
        # Get ATM option
        atm_call = opt.calls.iloc[(opt.calls['strike'] - stock_price).abs().idxmin()]
        
        # Store results
        results.append({
            "Symbol": symbol,
            "Expiration": closest_exp,
            "Days_To_Expiration": (closest_exp - datetime.now()).days,
            "Strike": atm_call['strike'],
            "Current_Price": stock_price,
            "Call_Price": atm_call['lastPrice'],
            "Implied_Volatility": round(atm_call['impliedVolatility'] * 100, 2),
            "Volume": atm_call['volume'],
            "Open_Interest": atm_call['openInterest']
        })
        
        print(f"Successfully processed {symbol}")
        
    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")

Processing MMM
Closest expiration for MMM: 2024-12-20
Error processing MMM: 'str' object has no attribute 'strftime'
Processing AOS
Closest expiration for AOS: 2024-12-20
Error processing AOS: 'str' object has no attribute 'strftime'
Processing ABT
Closest expiration for ABT: 2024-12-20
Error processing ABT: 'str' object has no attribute 'strftime'
Processing ABBV
Closest expiration for ABBV: 2024-12-20
Error processing ABBV: 'str' object has no attribute 'strftime'
Processing ACN
Closest expiration for ACN: 2024-12-20
Error processing ACN: 'str' object has no attribute 'strftime'
